# Spotify Music Popularity Capstone - Modeling

Greg Welliver   

In [2]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as plticker
import seaborn as sns
sns.set()
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats
from sklearn import tree, svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier


In [2]:
# load the data
df = pd.read_parquet('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')
df.head()

,Top100,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,Label_i! Records,Label_iDEAL,Label_independent,Label_mau5trap,Label_nterscope,"Label_will.i.am, A&M","Label_will.i.am, Cherrytree, Interscope, Dandyville",Label_~scape,Label_â„— 2019 Speakerbox Recordings Limited under,Label_å†¬æ¨¹ç¤¾
0,Y,0.030321,0.960526,0.057939,0.661,0.000000,0.0454,0.869679,0.059521,0.421389,...,0,0,0,0,0,0,0,0,0,0
1,Y,0.143574,0.853239,0.055064,0.806,0.000000,0.0771,0.843131,0.279917,0.382856,...,0,0,0,0,0,0,0,0,0,0
2,Y,0.037249,0.789474,0.083111,0.768,0.000002,0.5180,0.868416,0.247659,0.322836,...,0,0,0,0,0,0,0,0,0,0
3,Y,0.061847,0.860324,0.059530,0.700,0.000002,0.2440,0.834646,0.049740,0.410788,...,0,0,0,0,0,0,0,0,0,0
4,N,0.028213,0.733806,0.048665,0.640,0.000000,0.0990,0.812575,0.345473,0.669470,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#df = df.reset_index(drop=True)

In [4]:
df.shape

(255096, 24362)

### Split data into training and testing subsets

In [5]:
# Create the X and y matrices from the dataframe
X = df.drop(columns = ['Top100'])
y = df['Top100']

In [6]:
# from sklearn.model_selection import train_test_split; reserve 20% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=1)

### Model selection
- test out a couple different ones (making sure to try different parameters within each algorithm as well), and select the best one by cross-validation
- if you have a huge dataset, then whichever classification algorithm you use might not matter so much in terms of classification performance (so choose your algorithm based on speed or ease of use instead).
#### QUESTION: when/how does speed become an important consideration?
- If your training set is small, high bias/low variance classifiers (e.g., Naive Bayes) have an advantage over low bias/high variance classifiers (e.g., kNN), since the latter will overfit. But low bias/high variance classifiers start to win out as your training set grows (they have lower asymptotic error), since high bias classifiers aren’t powerful enough to provide accurate models.



#### Algorithms we will try:
- logistic regression
- decision tree
- k-nearest neighbor


#### QUESTION: try these?
- random forest
- svm
- naive bayes

In [3]:
# Logistic Regression
lr_clf = LogisticRegression(random_state = 1)
lr_clf.fit(X_train, y_train)
print('Accuracy on training set:',lr_clf.score(X_train,y_train))
print('Accuracy on test set:',lr_clf.score(X_test,y_test))

NameError: name 'X_train' is not defined

In [ ]:
# random forest regression
rf_clf = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_clf.fit(X_train, y_train)
print('Accuracy on training set:',rf_clf.score(X_train,y_train))
print('Accuracy on test set:',rf_clf.score(X_test,y_test))

In [ ]:
# decision tree
dec_clf = DecisionTreeClassifier()
dec_clf.fit(X_train, y_train)
print('Accuracy on training set:',dec_clf.score(X_train, y_train))
print('Accuracy on test set:',dec_clf.score(X_test, y_test))


In [ ]:
# xgboost
xgb_clf= XGBClassifier()
xgb_clf.fit(X_train, y_train)
rint('Accuracy score on train_data: ', accuracy_score(y_true = y_train, y_pred = xgb.predict(x_train).round()))
print('Accuracy score on test_data: ', accuracy_score(y_true = y_test, y_pred = xgb.predict(x_test).round()))

In [ ]:
# Naive Bayes
nb_clf= GaussianNB()
nb_clf.fit(X_train, y_train)
print('Accuracy on training set:',nb_clf.score(X_train,y_train))
print('Accuracy on test set:',nb_clf.score(X_test,y_test))

In [ ]:
# k-NN - note read that this tends not to work well on large datasets

classifiers = []

model1 = xgboost.XGBClassifier()
classifiers.append(model1)
model2 = svm.SVC()
classifiers.append(model2)
model3 = tree.DecisionTreeClassifier()
classifiers.append(model3)
model4 = RandomForestClassifier()
classifiers.append(model4)

for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred= clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))


#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

#### write to parquet
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_parquet(filepath, 
              engine = "pyarrow", 
              compression = None)

df = pd.read_parquet('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')